<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/model/Amazon_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_text

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
# nostri import 
import random
import pickle

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text #necessaria per hub.load

In [40]:
tf.test.is_gpu_available()

True

Pulizia del dataset da dei dati inutili per l'esperimento


In [41]:
df = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Amazon/Reviews.csv')
print(f"COLONNE:{df.columns}")
print(f"DIMENSIONE:{df.shape}")

COLONNE:Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
DIMENSIONE:(568454, 10)


In [42]:
del df['Id']
del df['ProductId']
del df['UserId']
del df['ProfileName']
del df['HelpfulnessNumerator']
del df['HelpfulnessDenominator']
del df['Time']

creazione colonna review data dall'unione del summary ed il testo

In [43]:
df['review'] = df['Summary']+df['Text']
del df['Summary']
del df['Text']
df.review.fillna("",inplace = True)
df.head()

,Score,review
0,5,Good Quality Dog FoodI have bought several of ...
1,1,Not as AdvertisedProduct arrived labeled as Ju...
2,4,"""Delight"" says it allThis is a confection that..."
3,2,Cough MedicineIf you are looking for the secre...
4,5,Great taffyGreat taffy at a great price. Ther...


Creazione review type al posto dello score, per avere tutte le frasi con valutazione minore di 4 come negative e le rimanneti come positive

In [44]:
df["review_type"] = df["Score"].apply(lambda x: "negative" if x < 4 else "positive")

In [45]:
del df['Score']

In [46]:
df.review_type.value_counts()

positive    443777
negative    124677
Name: review_type, dtype: int64

Bilanciamento delle sentence positive con quelle negative, in modo da avere un dataset bilanciato, fatto 50 50 poiche si hanno a disposizione molti dati 

In [47]:
positive_reviews = df[df.review_type == "positive"]
negative_reviews = df[df.review_type == "negative"]

In [48]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [49]:
positive_df = positive_reviews.sample(n=len(negative_reviews), random_state=RANDOM_SEED)
negative_df = negative_reviews

In [50]:
review_df = positive_df.append(negative_df).reset_index(drop=True)
review_df.shape
print(review_df.columns)

Index(['review', 'review_type'], dtype='object')


## **Modello unico**

Creazione one hot encoding per il review_type

In [51]:
from sklearn.preprocessing import OneHotEncoder

type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  review_df.review_type.to_numpy().reshape(-1, 1)
)

Creazione test e train set per il modello, generato tramite **train_test_split**,  70/30 con seed identico a quello usato per il campionamemto dei dati per il bilancaiamento del dataset

In [52]:
train_reviews, test_reviews, y_train, y_test =\
  train_test_split(
    review_df.review, 
    type_one_hot, 
    test_size=.3, 
    random_state=RANDOM_SEED
  )

In [53]:
print(train_reviews.shape)
print(test_reviews.shape)

(174547,)
(74807,)


Caricamento dell'embedding e creazione della funzione usata dal modello per usarlo all'interno dello stesso e non come preprocessing del dataset

In [54]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [55]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))#, 
    	#signature="default", as_dict=True)["default"]

In [56]:
model = keras.Sequential()

model.add(keras.layers.Input(shape=(1,), dtype=tf.string))
model.add(keras.layers.Lambda(UniversalEmbedding,output_shape=(512,))) # pre trained Convolutional Neural Net. 
model.add(keras.layers.Dense(units=256, activation='relu')) 
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'Embeddings/sharded_0:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_1:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_2:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_3:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_4:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_5:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_6:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_7:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_8:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_9:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_10:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_11:0' shape=(7530, 512) dtype=float32>
  <tf.Variab

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'Embeddings/sharded_0:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_1:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_2:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_3:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_4:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_5:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_6:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_7:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_8:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_9:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_10:0' shape=(7530, 512) dtype=float32>
  <tf.Variable 'Embeddings/sharded_11:0' shape=(7530, 512) dtype=float32>
  <tf.Variab

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 164,482
Trainable params: 164,482
Non-trainable params: 0
__________________________________________________

In [59]:
history = model.fit(
    train_reviews[:2500], y_train[:2500], 
    epochs=15, 
    batch_size=16, 
    validation_split=0.1, 
    verbose=1, 
    shuffle=True
)

Epoch 1/15
141/141 [==============================] - 18s 124ms/step - loss: 0.0124 - accuracy: 0.9969 - val_loss: 0.8216 - val_accuracy: 0.8200
Epoch 2/15
141/141 [==============================] - 17s 122ms/step - loss: 0.0070 - accuracy: 0.9996 - val_loss: 0.8822 - val_accuracy: 0.8200
Epoch 3/15
141/141 [==============================] - 17s 122ms/step - loss: 0.0081 - accuracy: 0.9987 - val_loss: 0.9902 - val_accuracy: 0.8040
Epoch 4/15
141/141 [==============================] - 18s 126ms/step - loss: 0.0148 - accuracy: 0.9978 - val_loss: 0.9622 - val_accuracy: 0.8120
Epoch 5/15
141/141 [==============================] - 17s 120ms/step - loss: 0.0066 - accuracy: 0.9982 - val_loss: 0.9841 - val_accuracy: 0.8040
Epoch 6/15
141/141 [==============================] - 17s 124ms/step - loss: 0.0092 - accuracy: 0.9973 - val_loss: 0.9219 - val_accuracy: 0.8120
Epoch 7/15
141/141 [==============================] - 17s 122ms/step - loss: 0.0355 - accuracy: 0.9893 - val_loss: 0.8970 - val_ac

Salva il modello per poterlo utilizzare per i vari test 


In [60]:
model.save('Amazon_base_model')

AssertionError: ignored

In [ ]:
model = keras.models.load_model('Amazon_base_model')

In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_reviews, y_test,batch_size=16)
print("test loss, test acc:", results)

Evaluate on test data
3690/4676 [======================>.......] - ETA: 1:37 - loss: 0.7762 - accuracy: 0.8356

## **OLD**
Da non usare in quanto l'embedding viene fatto esterno al modello e non dentro il modello stesso

In [ ]:
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

KeyboardInterrupt: ignored

In [ ]:
from sklearn.preprocessing import OneHotEncoder

type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  review_df.review_type.to_numpy().reshape(-1, 1)
)

In [ ]:
train_reviews, test_reviews, y_train, y_test =\
  train_test_split(
    review_df.review, 
    type_one_hot, 
    test_size=.1, 
    random_state=RANDOM_SEED
  )

In [ ]:
X_train = []
for r in tqdm(train_reviews):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(review_emb)

X_train = np.array(X_train)

In [ ]:
X_test = []
for r in tqdm(test_reviews):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_test.append(review_emb)

X_test = np.array(X_test)

In [ ]:
with open("/content/drive/Shareddrives/Deep Learning/datasets/Amazon/X_train.pkl", 'wb') as output:
  pickle.dump(X_train, output, protocol=4)

with open("/content/drive/Shareddrives/Deep Learning/datasets/Amazon/X_test.pkl", 'wb') as output:
  pickle.dump(X_test, output, protocol=4)

In [ ]:
with open("/content/drive/Shareddrives/Deep Learning/datasets/Amazon/X_train.pkl", 'rb') as output:
  print(output)
  X_train=pickle.load(output)

with open("/content/drive/Shareddrives/Deep Learning/datasets/Amazon/X_test.pkl", 'rb') as output:
  print(output)
  X_test=pickle.load(output)

In [ ]:
X_test.shape

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Dense(units=256, input_shape=(X_train.shape[1], ), activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(
    X_train, y_train, 
    epochs=15, 
    batch_size=16, 
    validation_split=0.1, 
    verbose=1, 
    shuffle=True
)

In [ ]:
train_reviews[180138]

In [ ]:
df.iloc[94752].review

In [ ]:
review_df.iloc[94752]

In [ ]:
y_test[0]

In [ ]:
frase="s.  His finicky culinary mandates are the most outrageous, unreasonable and demanding of almost any human I\'ve ever known.  His love of nuts is extremely persnickety, and he has been known to turn his nose up at some of the most expensive gourmet nut selections available.<br />Sunflower Food and Spice Co Honey Toasted Cashews gave me a first ever accomplishment in my gift-giving history with my sibling.  This year, instead of the usual  eh, not really that good actually.  or the one time in 1996 when I got a meh, they\'re ok I guess.  He said this:<br /><br />Best cashews I\'ve ever had.<br /><br />Truly, I can not tell you how significant a statement that is."

emb_frase=use(frase)

model.predict(emb_frase)